In [6]:
from functions import vectorize_y_ser

In [7]:
import string

In [8]:
import re

In [9]:
import datetime

In [10]:
import nltk
import string

In [11]:
import pandas as pd
import numpy as np

In [12]:
from collections import Counter

In [13]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [14]:
def vectorize_y_ser(ser):
    y = ser.copy()
    y.replace({'Op-Ed': 1,'News': 0}, inplace=True)
    return y

In [15]:
file = '/Users/djbetts/Desktop/opinion_classifier/data/archive/raw_2020.csv'

In [45]:
#combine print, type, text columns in the list
def printed(file):
    print_cols = ['_id','print_page','pub_date','keywords','type_of_material','lead_paragraph','text']    
    df = pd.read_csv(file, usecols=print_cols, index_col='_id', low_memory=False)
    df = df[df.print_page.isna() == False]
    df = df[~df.text.isna() == True]
    return df

In [53]:
#returns articles w/ 'United States Politics and Government' as a keyword
def return_pol_gov(row):
    list_dict = eval(row.keywords)
    for (dic) in (list_dict):
        if dic.get('value') == 'United States Politics and Government':
            return True

def keyword_filter(df, keyword='United States Politics and Government'):
    df[keyword] = df.apply(return_pol_gov, axis=1)
    df = df[df[keyword] == True]
    return df

# data

In [195]:
data = printed(file)

In [196]:
data.shape

(27752, 6)

In [197]:
%%time
df = keyword_filter(data)

CPU times: user 2.75 s, sys: 71.8 ms, total: 2.83 s
Wall time: 2.83 s


In [198]:
df.shape

(4585, 7)

In [199]:
df.columns

Index(['lead_paragraph', 'keywords', 'pub_date', 'type_of_material',
       'print_page', 'text', 'United States Politics and Government'],
      dtype='object')

In [200]:
df.shape

(4585, 7)

In [201]:
df['text'] = [text.replace(lead, "") for text, lead in zip(df.text[:].astype('str'), df.lead_paragraph[:].astype('str'))]

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
df.text[0][:500]

'But as you drew closer to the fifth floor, where Attorney General William Pelham Barr works out of a suite of offices, things started to loosen up. One assistant outside his conference room wore a mask, but the other did not. In the middle of the room, with its oil paintings and vaulted ceiling, the long central table had fewer chairs than you might expect, and an appropriate distance between them. But past the next door, inside the attorney general’s smaller personal office, Barr himself was al'

# remove quotations

In [203]:
#split document into article body and quotation
def text_parse(document):
    article = []
    quote_list = []
    
    open_quote = "“"
    close_quote = "”"
    
    close_split = document.split(close_quote)
    
    for string in close_split:

        quote = string.split(open_quote)
        article.append(quote.pop(0))
        quote_list += quote
    
    article = " ".join(article)
    quotation = " ".join(quote_list)
        
    return(article, quotation)

In [204]:
#split article and return quote
def return_article(document):
    article = []
    quote_list = []
    
    open_quote = "“"
    close_quote = "”"
    
    close_split = document.split(close_quote)
    
    for string in close_split:

        quote = string.split(open_quote)
        article.append(quote.pop(0))
        quote_list += quote
    
    article = " ".join(article)
    quotation = " ".join(quote_list)
    
    return(article)

In [205]:
#split article and return quote
def return_quote(document):
    article = []
    quote_list = []
    
    open_quote = "“"
    close_quote = "”"
    
    close_split = document.split(close_quote)
    
    for string in close_split:

        quote = string.split(open_quote)
        article.append(quote.pop(0))
        quote_list.append(quote)
    
    article = " ".join(article)
#    quotation = " ".join(quote_list)
        
    return(quote_list)

In [206]:
df['article'] = df.text.apply(return_article)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [207]:
df['article_text'] = df.text.apply(return_article)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [208]:
df['article'][0][:500]

'But as you drew closer to the fifth floor, where Attorney General William Pelham Barr works out of a suite of offices, things started to loosen up. One assistant outside his conference room wore a mask, but the other did not. In the middle of the room, with its oil paintings and vaulted ceiling, the long central table had fewer chairs than you might expect, and an appropriate distance between them. But past the next door, inside the attorney general’s smaller personal office, Barr himself was al'

In [209]:
df['quote_list'] = df.text.apply(return_quote)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [210]:
len(df['quote_list'][0])

178

# created keyword list

In [211]:
def keyword_string(row):
    
    dictionary_list = (eval(row))
    output = " "
    
    for dictionary in dictionary_list:
        dict_value = dictionary.get('value')
        output += (f'{dict_value} ')
    return output.split(" ")

In [212]:
df['keyword_list'] =  df.keywords.apply(keyword_string)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# remove punctuation

In [213]:
#string.punctuation
nytimes_punct = "”“"
open_quote = "“"
close_quote = "”"

In [214]:
#function that removes string.punctuation w/out the '?'
def punc_strip(document):
    for char in word_tokenize(document):
        if char in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—’':
            document = document.replace(char, " ")
        if char == "’":
            document = document.replace(char, "")
    return document

In [215]:
df['text'] = df.text.apply(punc_strip)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# remove capitalized words 

In [216]:
def remove_cap_words(row):
    row = row.split()
    for word in row:
        if word[0].isupper():
            row.remove(word)
            
    return " ".join(row)

In [217]:
df['text'] = df.text.apply(remove_cap_words)

/Users/djbetts/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [223]:
df.to_csv('test2020.csv')

# remove keywords

In [ ]:
df['text'] = [text.replace(lead, "") for article, keywords in zip(df.article[:].astype('str'), df.lead_paragraph[:].astype('str'))]

In [107]:
text = df.text[0]

In [123]:
article = df.article[0]

In [124]:
keywords = df.keyword_list[0]

In [129]:
article_list = article.split(" ")
for word in article_list:
    if word in keywords[1:-1]:
        article_list.remove(word)
print(article_list)

['But', 'as', 'you', 'drew', 'closer', 'to', 'the', 'fifth', 'floor,', 'where', 'Attorney', 'Pelham', 'Barr', 'works', 'out', 'a', 'suite', 'offices,', 'things', 'started', 'to', 'loosen', 'up.', 'One', 'assistant', 'outside', 'his', 'conference', 'room', 'wore', 'a', 'mask,', 'but', 'the', 'other', 'did', 'not.', 'In', 'the', 'middle', 'the', 'room,', 'with', 'its', 'oil', 'paintings', 'vaulted', 'ceiling,', 'the', 'long', 'central', 'table', 'had', 'fewer', 'chairs', 'than', 'you', 'might', 'expect,', 'an', 'appropriate', 'distance', 'between', 'them.', 'But', 'past', 'the', 'next', 'door,', 'inside', 'the', 'attorney', 'general’s', 'smaller', 'personal', 'office,', 'Barr', 'himself', 'was', 'also', 'mask-free.', 'Turning', 'around', 'to', 'greet', 'his', 'visitors,', 'he', 'moved', 'into', 'the', 'middle', 'a', 'wide', 'circle', 'four', 'chairs', 'arranged', 'in', 'front', 'his', 'desk.Now', 'nearing', 'the', 'end', 'his', 'career,', 'Barr', 'did', 'not', 'take', 'his', 'current', '

In [127]:
article

'But as you drew closer to the fifth floor, where Attorney General William Pelham Barr works out of a suite of offices, things started to loosen up. One assistant outside his conference room wore a mask, but the other did not. In the middle of the room, with its oil paintings and vaulted ceiling, the long central table had fewer chairs than you might expect, and an appropriate distance between them. But past the next door, inside the attorney general’s smaller personal office, Barr himself was also mask-free. Turning around to greet his visitors, he moved into the middle of a wide circle of four chairs arranged in front of his desk.Now nearing the end of his career, Barr did not take his current job for the glory. He had already been attorney general once, in President George H.W. Bush’s administration, winning him a reputation as a wise old man — a reputation that, in the eyes of some, his tenure in the Trump administration has tarnished. Nor is he doing it for the money. His time in 

In [387]:
for word in document.split():
    word_upper = []
    if word[0].isupper():
        word_upper.append(word) 
    if word in word_upper:
        document = document.replace(word, "")

In [2]:
print(document)

NameError: name 'document' is not defined

In [3]:
%%time
def wordnet_tokenize(doc):
    wordnet = WordNetLemmatizer()
    return  [wordnet.lemmatize(word) for word in word_tokenize(doc.lower())]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [30]:
%%time
df = data.copy()

CPU times: user 9.24 ms, sys: 2.26 ms, total: 11.5 ms
Wall time: 10.9 ms


In [38]:
documents = list(df.text)

In [110]:
sample = documents[0]

In [111]:
type(sample)

str

# fix contractions

In [50]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [55]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [56]:
document = expand_contractions(document)

In [60]:
"’" == "’"

True

In [57]:
document

'WASHINGTON — Federal prosecutors rebuffed a judge’s order to release by Friday highly classified transcripts of discussions that Michael T. Flynn, the president’s former national security adviser, had with the Russian ambassador during the presidential transition.The transcripts between Mr. Flynn and Sergey I. Kislyak, formerly Russia’s top diplomat in the United States, were expected to show that they talked in December 2016 about sanctions that the Obama administration had just imposed on Russia. Mr. Flynn initially denied those exchanges about sanctions both to Trump administration officials and the F.B.I. in the weeks after the discussions.The conversations prompted concerns among senior Obama administration officials about whether the Trump transition team was flouting norms about holding off on making policy until after taking office. The phone calls were also at the center of the scandal that eventually prompted Mr. Flynn’s ouster just weeks into President Trump’s term.The orde

# stemming/lemmatization

In [30]:
document = df.text.iloc[0]

In [22]:
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

In [23]:
%%time
def snowball_tokenize(doc):
    snowball = SnowballStemmer('english')
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [24]:
%%time
def wordnet_tokenize(doc):
    wordnet = WordNetLemmatizer()
    return  [wordnet.lemmatize(word) for word in word_tokenize(doc.lower())]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [25]:
test = word_tokenize(document.lower())

In [37]:
#document.lower()

In [42]:
document = df.text.iloc[0]

In [44]:
document = snowball_tokenize(document)
document

['washington',
 '—',
 'feder',
 'prosecutor',
 'rebuf',
 'a',
 'judg',
 '’',
 's',
 'order',
 'to',
 'releas',
 'by',
 'friday',
 'high',
 'classifi',
 'transcript',
 'of',
 'discuss',
 'that',
 'michael',
 't.',
 'flynn',
 ',',
 'the',
 'presid',
 '’',
 's',
 'former',
 'nation',
 'secur',
 'advis',
 ',',
 'had',
 'with',
 'the',
 'russian',
 'ambassador',
 'dure',
 'the',
 'presidenti',
 'transition.th',
 'transcript',
 'between',
 'mr.',
 'flynn',
 'and',
 'sergey',
 'i.',
 'kislyak',
 ',',
 'former',
 'russia',
 '’',
 's',
 'top',
 'diplomat',
 'in',
 'the',
 'unit',
 'state',
 ',',
 'were',
 'expect',
 'to',
 'show',
 'that',
 'they',
 'talk',
 'in',
 'decemb',
 '2016',
 'about',
 'sanction',
 'that',
 'the',
 'obama',
 'administr',
 'had',
 'just',
 'impos',
 'on',
 'russia',
 '.',
 'mr.',
 'flynn',
 'initi',
 'deni',
 'those',
 'exchang',
 'about',
 'sanction',
 'both',
 'to',
 'trump',
 'administr',
 'offici',
 'and',
 'the',
 'f.b.i',
 '.',
 'in',
 'the',
 'week',
 'after',
 '

In [39]:
document = punc_strip(document)
document

In [58]:
stopword_list = nltk.corpus.stopwords.words('english')

In [59]:
stopword_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each